**0. Preliminary Works:** how big is the file,number of lines, etc.<br>
**1. Prepare sample file:** create small file to work with.<br>
**2. Study the Data**<br>
**3. Select Columns of Interest**<br>
**4. Data Cleaning**<br>
    4.1. What to do with NaN<br>
    4.2. Make an action plan<br>
**5. Visualization**<br>
**6. Modelling**<br>
**7. Evaluation**<br>
**FINAL: Adjust the code to work with Big data**<br>
    5.1. Create an ITERATOR from the data base and define CHUNK size <br>
    5.2. Test Big Data approach on a sample

# 0. Premilinary Works
How big is the file, number of rows, etc.

In [1]:
!ls -Rl
# 2019_Yellow_Taxi_Trip_Data.csv is 4.243.651.749  bytes (7,63 GB)

.:
total 14996
-rw-rw-rw- 1 angelrps angelrps  143136 Apr  2 16:55 04-group_by_inclass.ipynb
drwxrwxrwx 1 angelrps angelrps     512 Apr  1 18:31 Data
-rwxrwxrwx 1 angelrps angelrps  249641 Apr  2 19:30 Data_Analysis.ipynb
-rw-rw-rw- 1 angelrps angelrps   10144 Apr  1 12:33 HowToTackleDataScienceChallenge.ipynb
---------- 1 angelrps angelrps    9058 Apr  1 19:59 Schedule.xlsx
-rwxrwxrwx 1 angelrps angelrps 3527394 Apr  2 17:37 TSA.ipynb

./Data:
total 15113920
-rwxrwxrwx 1 angelrps angelrps 10532305815 Mar 25 13:15  2017_Yellow_Taxi_Trip_Data.csv
-rwxrwxrwx 1 angelrps angelrps  4243651749 Apr  1 19:51  2019_Yellow_Taxi_Trip_Data.csv
-rwxrwxrwx 1 angelrps angelrps       73972 Apr  1 18:56  data_dictionary_trip_records_yellow.pdf
-rwxrwxrwx 1 angelrps angelrps          83 Mar 25 12:21  reserva_paradas_taxis.txt
-rwxrwxrwx 1 angelrps angelrps       51576 Mar 25 11:59 'reserva_paradas_taxis_NO SIRVE NO HAY DATOS SUFICIENTES POR AÑ.csv'
-rwxrwxrwx 1 angelrps angelrps     9280017 Apr  1 14:01

In [2]:
# count lines
!wc -l ./Data/2019_Yellow_Taxi_Trip_Data.csv
# the file has 43.801.887 lines

43801887 ./Data/2019_Yellow_Taxi_Trip_Data.csv


In [3]:
!head -5 ./Data/2019_Yellow_Taxi_Trip_Data.csv
# There is a header
# And the separator is ';'
# Columns 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' should be imported as datetime

﻿VendorID;tpep_pickup_datetime;tpep_dropoff_datetime;passenger_count;trip_distance;RatecodeID;store_and_fwd_flag;PULocationID;DOLocationID;payment_type;fare_amount;extra;mta_tax;tip_amount;tolls_amount;improvement_surcharge;total_amount;congestion_surcharge
2;08/31/2019 05:42:04 PM;08/31/2019 06:18:38 PM;1;6.24;1;N;163;194;2;26;0;0.5;0;6.12;0.3;35.42;2.5
2;08/31/2019 05:01:41 PM;08/31/2019 05:21:30 PM;5;3.18;1;N;142;113;2;15;0;0.5;0;0;0.3;18.3;2.5
2;08/31/2019 05:10:42 PM;08/31/2019 05:41:00 PM;1;13.69;1;N;132;70;2;38;0;0.5;0;0;0.3;38.8;0
2;08/31/2019 05:45:15 PM;09/01/2019 12:00:00 AM;1;13.56;4;N;70;265;2;41;0;0.5;0;6.12;0.3;47.92;0


# 1. Prepare sample file
I will take ~ 10.000 for the sample file out of the 113 million lines of the file

In [4]:
import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [7]:
# TAKING RANDOM SAMPLE OF ~5.000 ROWS
# I will also parse the datetime columns to the right object type
p = 0.0001 # take 0.01 percent of total rows
dsample = pd.read_csv('./Data/2019_Yellow_Taxi_Trip_Data.csv',sep=';',header=0,
                      parse_dates = {'pickup_datetime':['tpep_pickup_datetime'],'dropoff_datetime':['tpep_dropoff_datetime']},
                        skiprows=lambda i: i>0 and random.random() > p)

In [8]:
dsample.shape

(4359, 18)

In [ ]:
# CREATE SAMPLE DATASET WITH 100 LINES WITH BASH

# it may or it may not have header
#!shuf -n 100 ./Data/2017_Yellow_Taxi_Trip_Data.csv > ./Data/sample100.csv

# without header
#!tail -n +2 ./Data/2017_Yellow_Taxi_Trip_Data.csv | shuf -n 100 > ./Data/sample100.csv

# 2. Study the Data
```python
b.shape         #check nº of rows and columns
list(b.columns) #check white spaces in column names
b.sample()      #check random sample
                #ask for the data dictionary and study all fields
b.info()        #check data types and spot nulls
b.describe()    #check descriptive statistics
```

In [9]:
dsample.shape, list(dsample.columns)
# I get the expected number of rows
# Column names have no spaces

((4359, 18),
 ['pickup_datetime',
  'dropoff_datetime',
  'VendorID',
  'passenger_count',
  'trip_distance',
  'RatecodeID',
  'store_and_fwd_flag',
  'PULocationID',
  'DOLocationID',
  'payment_type',
  'fare_amount',
  'extra',
  'mta_tax',
  'tip_amount',
  'tolls_amount',
  'improvement_surcharge',
  'total_amount',
  'congestion_surcharge'])

In [10]:
dsample.sample(5)
# I have checked the data dictionary "data_dictionary_trip_records_yellow.pdf"
# values on this sample seem to make sense

,pickup_datetime,dropoff_datetime,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1712,2019-11-15 08:28:45,2019-11-15 08:40:37,1.0,1.0,2.10,1.0,N,68,142,1.0,10.0,2.5,0.5,2.65,0.0,0.3,15.95,2.5
36,2019-09-02 20:23:47,2019-09-02 20:34:01,2.0,1.0,2.08,1.0,N,161,249,1.0,9.0,0.5,0.5,2.56,0.0,0.3,15.36,2.5
1436,2019-11-03 17:54:09,2019-11-03 18:17:51,2.0,1.0,2.63,1.0,N,164,144,2.0,15.5,0.0,0.5,0.00,0.0,0.3,18.80,2.5
414,2019-09-20 02:38:04,2019-09-20 02:51:04,1.0,1.0,4.90,1.0,N,231,17,1.0,16.0,3.0,0.5,5.90,0.0,0.3,25.70,2.5
3679,2019-02-07 17:21:29,2019-02-07 17:34:45,2.0,2.0,1.08,1.0,N,231,87,1.0,9.5,1.0,0.5,2.76,0.0,0.3,16.56,2.5


In [11]:
dsample.info()
# there are no null values
# data types seem correct

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4359 entries, 0 to 4358
Data columns (total 18 columns):
pickup_datetime          4359 non-null datetime64[ns]
dropoff_datetime         4359 non-null datetime64[ns]
VendorID                 4343 non-null float64
passenger_count          4343 non-null float64
trip_distance            4359 non-null float64
RatecodeID               4343 non-null float64
store_and_fwd_flag       4343 non-null object
PULocationID             4359 non-null int64
DOLocationID             4359 non-null int64
payment_type             4343 non-null float64
fare_amount              4359 non-null float64
extra                    4359 non-null float64
mta_tax                  4359 non-null float64
tip_amount               4359 non-null float64
tolls_amount             4359 non-null float64
improvement_surcharge    4359 non-null float64
total_amount             4359 non-null float64
congestion_surcharge     3871 non-null float64
dtypes: datetime64[ns](2), float64(13)

In [12]:
dsample.describe()
# I see that:
# There are trips with 0 'trip_distance'
# There are trips with negative 'fare_amount'

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,4343.000000,4343.000000,4359.000000,4343.000000,4359.000000,4359.000000,4343.000000,4359.000000,4359.000000,4359.000000,4359.000000,4359.000000,4359.000000,4359.000000,3871.000000
mean,1.646558,1.568731,2.907733,1.061478,164.160587,161.299610,1.288741,13.151991,1.012618,0.493232,2.172645,0.364671,0.298211,18.694457,2.104107
std,0.504806,1.196613,3.687978,0.390639,66.016887,70.189697,0.481323,11.750746,1.215511,0.068670,2.621229,1.581569,0.032084,14.467177,0.926852
min,1.000000,0.000000,0.000000,1.000000,4.000000,1.000000,1.000000,-20.500000,-1.000000,-0.500000,-0.760000,0.000000,-0.300000,-23.800000,-2.500000
25%,1.000000,1.000000,0.965000,1.000000,125.000000,112.000000,1.000000,6.500000,0.000000,0.500000,0.000000,0.000000,0.300000,10.800000,2.500000
50%,2.000000,1.000000,1.600000,1.000000,162.000000,162.000000,1.000000,9.500000,0.500000,0.500000,1.850000,0.000000,0.300000,14.300000,2.500000
75%,2.000000,2.000000,3.080000,1.000000,234.000000,234.000000,2.000000,15.000000,2.500000,0.500000,2.860000,0.000000,0.300000,20.300000,2.500000
max,4.000000,6.000000,26.290000,5.000000,265.000000,265.000000,4.000000,225.000000,7.000000,0.500000,28.770000,19.500000,0.300000,225.800000,2.500000


#### Check trip_distance = 0
**CONCLUSION:**<br>
I will remove all trips with 'trip_distance' < 0.06 (100 meters) because I consider them either measurement errors or non representative data.

In [13]:
dsample['ZeroDistCount'] = 1
dsample.groupby('trip_distance', as_index = False)['ZeroDistCount'].sum()
# there are 46 rows in this sample with 0 distance and few others with distances under 100 meters (0.06 miles)

,trip_distance,ZeroDistCount
0,0.00,46
1,0.01,2
2,0.02,4
3,0.04,2
4,0.05,1
5,0.09,2
6,0.10,2
7,0.16,4
8,0.18,1
9,0.19,1


#### Check negative fare_amount values
CONCLUSION:<br>
There are 6 rows with negative fare_amount and one row with 0.01. I will remove these rows because I consider them measurement errors.

In [15]:
dsample['FareCount'] = 1
dsample[dsample['fare_amount']<1].groupby('fare_amount', as_index = False)['FareCount'].sum()
# There are 12 rows in this sample with negative fare_amount and one row with 0.01.

,fare_amount,FareCount
0,-20.50,1
1,-12.00,1
2,-6.00,3
3,-4.50,1
4,-3.00,1
5,-2.50,5
6,0.00,1
7,0.01,2


In [16]:
# REMOVE 'trip_distance' < 0.06 AND 'fare_amount' < 1
dsample = dsample[(dsample['trip_distance']>0.06) & (dsample['fare_amount']>=1)]

In [17]:
# Groupby DATE and sort values to check if all the data is from 2019
dsample['DayCount'] = 1
d = dsample[['pickup_datetime', 'DayCount']].groupby([dsample['pickup_datetime'].dt.date]).sum().reset_index().sort_values('pickup_datetime')
d.head(), d.tail()
# I verify that all data belongs to the same year
# Also verify that there is data from January to December

(  pickup_datetime  DayCount
 0      2019-01-01        20
 1      2019-01-02        20
 2      2019-01-03        17
 3      2019-01-04        23
 4      2019-01-05        27,     pickup_datetime  DayCount
 184      2019-12-27        19
 185      2019-12-28        16
 186      2019-12-29        24
 187      2019-12-30        17
 188      2019-12-31        15)